In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import tqdm
import annotation, misc, hyspec_io, image_render
import skimage.exposure
import tensorflow as tf

In [3]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [4]:
# Paths
pca_data_model_file = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/M_PCA_Model/20210825_OlbergAreaA_PCA_data_and_model.npz')
spectra_dir_train = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4c_Rad_Georef_SGC_Spectra/TrainingAndValidationDatasets')
spectra_dir_test = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4c_Rad_Georef_SGC_Spectra/TestDatasets')
class_dict_file = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4c_Rad_Georef_SGC_Spectra/spectra_classes.json')
pca_dataset_train_path = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/5b_Rad_Georef_SGC_PCA-spectra/20210825_Olberg_PCA_TrainValDataset.npz')
pca_dataset_test_path = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/5b_Rad_Georef_SGC_PCA-spectra/20210825_Olberg_PCA_TestDataset.npz')

In [5]:
# Load PCA model
with np.load(pca_data_model_file) as npz_files:
    X_mean = npz_files['spectra_mean']
    X_std = npz_files['spectra_std']
    W_pca = npz_files['pca_components']

In [6]:
# Get class dict
class_dict = annotation.read_class_dict(class_dict_file)

In [7]:
# Get list of datasets
dataset_paths = [path for path in spectra_dir_train.glob('*') if path.is_dir()]

# Iterate over datasets and merge numpy arrays
pca_list = []
labels_list = []
for dataset_path in tqdm.tqdm(dataset_paths):
    dataset = tf.data.experimental.load(str(dataset_path))
    spectra,labels = next(dataset.batch(dataset.cardinality()).as_numpy_iterator())
    spectra_pca = ((spectra - X_mean)/X_std) @ W_pca
    pca_list.append(spectra_pca)
    labels_list.append(labels)
pca_scores = np.concatenate(pca_list)
labels = np.concatenate(labels_list)

# Save data as numpy array 
np.savez(pca_dataset_train_path,
         pca_scores=pca_scores, 
         labels=labels, 
         class_dict=class_dict)

100%|████████████████████████████████████████████████████████| 16/16 [00:26<00:00,  1.63s/it]


In [8]:
# Get list of datasets
dataset_paths = [path for path in spectra_dir_test.glob('*') if path.is_dir()]

# Iterate over datasets and merge numpy arrays
pca_list = []
labels_list = []
for dataset_path in tqdm.tqdm(dataset_paths):
    dataset = tf.data.experimental.load(str(dataset_path))
    spectra,labels = next(dataset.batch(dataset.cardinality()).as_numpy_iterator())
    spectra_pca = ((spectra - X_mean)/X_std) @ W_pca
    pca_list.append(spectra_pca)
    labels_list.append(labels)
pca_scores = np.concatenate(pca_list)
labels = np.concatenate(labels_list)

# Save data as numpy array 
np.savez(pca_dataset_test_path,
         pca_scores=pca_scores, 
         labels=labels, 
         class_dict=class_dict)

100%|██████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.12s/it]
